
# Unveiling Hidden Patterns: A Market Basket Analysis using Apriori Algorithm and Association Rule Mining



<img src="https://miro.medium.com/v2/resize:fit:1400/0*oZ_5b6y4WmIwOIlJ.png" alt="alttext">

### Motivation
I aimed to run association analysis in Python using the apriori algorithm to derive rules like {A} -> {B}. Unfortunately, standard Python machine learning libraries lacked a suitable implementation for my large dataset, consisting of 32 million records with 3.2 million unique orders and 50K unique items. Writing my own implementation using apriori was the solution. Although I faced memory issues, I discovered the magic of Python generators, enabling efficient handling of large datasets and allowing me to continue the analysis successfully.


### Python Generators

In a nutshell, a generator is a special type of function that returns an iterable sequence of items.  However, unlike regular functions which return all the values at once (eg: returning all the elements of a list), a generator <i>yields</i> one value at a time.  To get the next value in the set, we must ask for it - either by explicitly calling the generator's built-in "next" method, or implicitly via a for loop.  This is a great property of generators because it means that we don't have to store all of the values in memory at once.  We can load and process one value at a time, discard when finished and move on to process the next value.  This feature makes generators perfect for creating item pairs and counting their frequency of co-occurence.  Here's a concrete example of what we're trying to accomplish:  

1. Get all possible item pairs for a given order 
       eg:  order 1:  apple, egg, milk   -->  item pairs: {apple, egg}, {apple, milk}, {egg, milk}
            order 2:  egg, milk          -->  item pairs: {egg, milk}
            
2. Count the number of times each item pair appears
       eg: {apple, egg}: 1
           {apple, milk}: 1
           {egg, milk}: 2

Here's the generator that implements the above tasks:

In [78]:
import numpy as np
from itertools import combinations, groupby
from collections import Counter

# Sample data
orders = np.array([[1,'apple'], [1,'egg'], [1,'milk'], [2,'egg'], [2,'milk']], dtype=object)

# Generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    
    # For each order, generate a list of items in that order
    for order_id, order_object in groupby(orders, lambda x: x[0]):
        item_list = [item[1] for item in order_object]      
    
        # For each item list, generate item pairs, one at a time
        for item_pair in combinations(item_list, 2):
            yield item_pair                                      


# Counter iterates through the item pairs returned by our generator and keeps a tally of their occurrence
Counter(get_item_pairs(orders))


Counter({('apple', 'egg'): 1, ('apple', 'milk'): 1, ('egg', 'milk'): 2})

<i>get_item_pairs()</i> generates a list of items for each order and produces item pairs for that order, one pair at a time.  The first item pair is passed to Counter which keeps track of the number of times an item pair occurs.  The next item pair is taken, and again, passed to Counter.  This process continues until there are no more item pairs left.  With this approach, we end up not using much memory as item pairs are discarded after the count is updated.

### Apriori Algorithm 
Apriori is an algorithm used to identify frequent item sets (in our case, item pairs).  It does so using a "bottom up" approach, first identifying individual items that satisfy a minimum occurence threshold. It then extends the item set, adding one item at a time and checking if the resulting item set still satisfies the specified threshold.  The algorithm stops when there are no more items to add that meet the minimum occurrence requirement.  Here's an example of apriori in action, assuming a minimum occurence threshold of 3:


    order 1: apple, egg, milk  
    order 2: carrot, milk  
    order 3: apple, egg, carrot
    order 4: apple, egg
    order 5: apple, carrot

    
    Iteration 1:  Count the number of times each item occurs   
    item set      occurrence count    
    {apple}              4   
    {egg}                3   
    {milk}               2   
    {carrot}             2   

    {milk} and {carrot} are eliminated because they do not meet the minimum occurrence threshold.


    Iteration 2: Build item sets of size 2 using the remaining items from Iteration 1 
                 (ie: apple, egg)  
    item set           occurence count  
    {apple, egg}             3  

    Only {apple, egg} remains and the algorithm stops since there are no more items to add.
   
   
If we had more orders and items, we can continue to iterate, building item sets consisting of more than 2 elements.  For the problem we are trying to solve (ie: finding relationships between pairs of items), it suffices to implement apriori to get to item sets of size 2.

### Association Rules Mining
Once the item sets have been generated using apriori, we can start mining association rules.  Given that we are only looking at item sets of size 2, the association rules we will generate will be of the form {A} -> {B}.  One common application of these rules is in the domain of recommender systems, where customers who purchased item A are recommended item B.

Here are 3 key metrics to consider when evaluating association rules:

1. <b>support</b>  
    This is the percentage of orders that contains the item set. In the example above, there are 5 orders in total 
    and {apple,egg} occurs in 3 of them, so: 
       
                    support{apple,egg} = 3/5 or 60%
        
    The minimum support threshold required by apriori can be set based on knowledge of your domain.  In this 
    grocery dataset for example, since there could be thousands of distinct items and an order can contain 
    only a small fraction of these items, setting the support threshold to 0.01% may be reasonable.<br><br><br>
    
2. <b>confidence</b>  
    Given two items, A and B, confidence measures the percentage of times that item B is purchased, given that 
    item A was purchased. This is expressed as:
       
                    confidence{A->B} = support{A,B} / support{A}   
                    
    Confidence values range from 0 to 1, where 0 indicates that B is never purchased when A is purchased, and 1 
    indicates that B is always purchased whenever A is purchased.  Note that the confidence measure is directional.     This means that we can also compute the percentage of times that item A is purchased, given that item B was 
    purchased:
       
                    confidence{B->A} = support{A,B} / support{B}    
                    
    In our example, the percentage of times that egg is purchased, given that apple was purchased is:  
       
                    confidence{apple->egg} = support{apple,egg} / support{apple}
                                           = (3/5) / (4/5)
                                           = 0.75 or 75%

    A confidence value of 0.75 implies that out of all orders that contain apple, 75% of them also contain egg.  Now, 
    we look at the confidence measure in the opposite direction (ie: egg->apple): 
       
                    confidence{egg->apple} = support{apple,egg} / support{egg}
                                           = (3/5) / (3/5)
                                           = 1 or 100%  
                                           
    Here we see that all of the orders that contain egg also contain apple.  But, does this mean that there is a 
    relationship between these two items, or are they occurring together in the same orders simply by chance?  To 
    answer this question, we look at another measure which takes into account the popularity of <i>both</i> items.<br><br><br>  
    
3. <b>lift</b>  
    Given two items, A and B, lift indicates whether there is a relationship between A and B, or whether the two items 
    are occuring together in the same orders simply by chance (ie: at random).  Unlike the confidence metric whose 
    value may vary depending on direction (eg: confidence{A->B} may be different from confidence{B->A}), 
    lift has no direction. This means that the lift{A,B} is always equal to the lift{B,A}: 
       
                    lift{A,B} = lift{B,A} = support{A,B} / (support{A} * support{B})   
    
    In our example, we compute lift as follows:
    
         lift{apple,egg} = lift{egg,apple} = support{apple,egg} / (support{apple} * support{egg})
                         = (3/5) / (4/5 * 3/5) 
                         = 1.25    
               
    One way to understand lift is to think of the denominator as the likelihood that A and B will appear in the same 
    order if there was <i>no</i> relationship between them. In the example above, if apple occurred in 80% of the
    orders and egg occurred in 60% of the orders, then if there was no relationship between them, we would 
    <i>expect</i> both of them to show up together in the same order 48% of the time (ie: 80% * 60%).  The numerator, 
    on the other hand, represents how often apple and egg <i>actually</i> appear together in the same order.  In 
    this example, that is 60% of the time.  Taking the numerator and dividing it by the denominator, we get to how 
    many more times apple and egg actually appear in the same order, compared to if there was no relationship between     them (ie: that they are occurring together simply at random).  
    
    In summary, lift can take on the following values:
    
        * lift = 1 implies no relationship between A and B. 
          (ie: A and B occur together only by chance)
      
        * lift > 1 implies that there is a positive relationship between A and B.
          (ie:  A and B occur together more often than random)
    
        * lift < 1 implies that there is a negative relationship between A and B.
          (ie:  A and B occur together less often than random)
        
    In our example, apple and egg occur together 1.25 times <i>more</i> than random, so we conclude that there exists 
    a positive relationship between them.
   
Armed with knowledge of apriori and association rules mining, let's dive into the data and code to see what relationships we unravel!

### Input Dataset
Instacart, an online grocer, has graciously made some of their datasets accessible to the public.  The order and product datasets that we will be using can be downloaded from the link below, along with the data dictionary:

“The Instacart Online Grocery Shopping Dataset 2017”, Accessed from https://www.kaggle.com/datasets/aslanahmedov/market-basket-analysis.


In [79]:
import pandas as pd
import numpy as np
import sys
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from IPython.display import display
import warnings


In [80]:
# Function that returns the size of an object in MB
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

### Part 1:  Data Preparation

#### A. Load order  data

In [81]:
orders = pd.read_csv('C:/Users/susha/OneDrive/Desktop/Data Warehouse/Aprioro_data.csv')
print('orders -- dimensions: {0};   size: {1}'.format(orders.shape, size(orders)))
display(orders.head())

orders -- dimensions: (22676, 7);   size: 5.86 MB


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6.0,01.12.2010 08:26,255.0,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6.0,01.12.2010 08:26,339.0,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8.0,01.12.2010 08:26,275.0,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,01.12.2010 08:26,339.0,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6.0,01.12.2010 08:26,339.0,17850.0,United Kingdom


#### B. Convert order data into format expected by the association rules function

In [82]:
orders['Itemname'] = orders['Itemname'].str.strip()
orders.dropna(axis = 0, subset = ['BillNo'], inplace = True)
orders['BillNo'] = orders['BillNo'].astype('str')
# orders = orders[~orders['BillNo'].str.contains('C')]
orders

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6.0,01.12.2010 08:26,255.0,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6.0,01.12.2010 08:26,339.0,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8.0,01.12.2010 08:26,275.0,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,01.12.2010 08:26,339.0,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6.0,01.12.2010 08:26,339.0,17850.0,United Kingdom
...,...,...,...,...,...,...,...
22671,538184,CALENDAR PAPER CUT DESIGN,6.0,10.12.2010 10:21,295.0,17880.0,United Kingdom
22672,538185,15CM CHRISTMAS GLASS BALL 20 LIGHTS,10.0,10.12.2010 10:22,795.0,16265.0,United Kingdom
22673,538185,VINTAGE SNAP CARDS,24.0,10.12.2010 10:22,85.0,16265.0,United Kingdom
22674,538185,VINTAGE HEADS AND TAILS CARD GAME,24.0,10.12.2010 10:22,125.0,16265.0,United Kingdom


#### C. Perform one-hot encoding to prepare the data for Apriori algorithm

In [83]:
basket = (orders.groupby(['BillNo', 'Itemname'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('BillNo'))
basket


Itemname,*Boombox Ipod Classic,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,...,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,amazon,check,damages,faulty
BillNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### D. Convert the values to binary (0 or 1) for Apriori

In [84]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units)
basket_sets

Itemname,*Boombox Ipod Classic,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,...,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,amazon,check,damages,faulty
BillNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
538182,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
538183,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### E. Apply Apriori algorithm to find frequent itemsets

In [95]:
##Ignore this line
warnings.filterwarnings('ignore', message='DataFrames with non-bool types result in worse computational performance.*')

frequent_itemsets = apriori(basket_sets, min_support = 0.03, use_colnames = True)
rules = association_rules(frequent_itemsets, metric = "lift", min_threshold=1)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(60 CAKE CASES VINTAGE CHRISTMAS),(PAPER CHAIN KIT 50'S CHRISTMAS),0.061466,0.117021,0.034279,0.557692,4.765734,0.027086,1.996300
1,(PAPER CHAIN KIT 50'S CHRISTMAS),(60 CAKE CASES VINTAGE CHRISTMAS),0.117021,0.061466,0.034279,0.292929,4.765734,0.027086,1.327356
2,(PAPER CHAIN KIT VINTAGE CHRISTMAS),(60 CAKE CASES VINTAGE CHRISTMAS),0.098109,0.061466,0.039007,0.397590,6.468489,0.032977,1.557967
3,(60 CAKE CASES VINTAGE CHRISTMAS),(PAPER CHAIN KIT VINTAGE CHRISTMAS),0.061466,0.098109,0.039007,0.634615,6.468489,0.032977,2.468334
4,(SET OF 20 VINTAGE CHRISTMAS NAPKINS),(60 CAKE CASES VINTAGE CHRISTMAS),0.065012,0.061466,0.034279,0.527273,8.578322,0.030283,1.985361
5,(60 CAKE CASES VINTAGE CHRISTMAS),(SET OF 20 VINTAGE CHRISTMAS NAPKINS),0.061466,0.065012,0.034279,0.557692,8.578322,0.030283,2.113886
6,(60 TEATIME FAIRY CAKE CASES),(PACK OF 72 RETROSPOT CAKE CASES),0.042553,0.066194,0.030733,0.722222,10.910714,0.027916,3.361702
7,(PACK OF 72 RETROSPOT CAKE CASES),(60 TEATIME FAIRY CAKE CASES),0.066194,0.042553,0.030733,0.464286,10.910714,0.027916,1.787234
8,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.065012,0.042553,0.031915,0.490909,11.536364,0.029148,1.880699
9,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.042553,0.065012,0.031915,0.750000,11.536364,0.029148,3.739953


#### Feel free to change the rules criteria as per your requirement

In [89]:
rules[(rules['lift'] >= 6) &
       (rules['confidence'] >= 0.75) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
9,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.042553,0.065012,0.031915,0.750000,11.536364,0.029148,3.739953
11,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.053191,0.065012,0.041371,0.777778,11.963636,0.037913,4.207447
36,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.042553,0.044917,0.033097,0.777778,17.315789,0.031186,4.297872
115,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.041371,0.079196,0.031915,0.771429,9.740725,0.028638,4.028517
125,(LARGE POPCORN HOLDER),(SMALL POPCORN HOLDER),0.042553,0.075650,0.037825,0.888889,11.750000,0.034606,8.319149
136,(POPPY'S PLAYHOUSE KITCHEN),(POPPY'S PLAYHOUSE BEDROOM),0.043735,0.046099,0.036643,0.837838,18.174636,0.034627,5.882388
137,(POPPY'S PLAYHOUSE BEDROOM),(POPPY'S PLAYHOUSE KITCHEN),0.046099,0.043735,0.036643,0.794872,18.174636,0.034627,4.661791
181,"(HAND WARMER RED RETROSPOT, HAND WARMER OWL DE...",(HAND WARMER SCOTTY DOG DESIGN),0.043735,0.092199,0.034279,0.783784,8.501040,0.030247,4.198582
186,"(WHITE HANGING HEART T-LIGHT HOLDER, KNITTED U...",(RED WOOLLY HOTTIE WHITE HEART.),0.046099,0.105201,0.037825,0.820513,7.799481,0.032975,4.985309
187,"(WHITE HANGING HEART T-LIGHT HOLDER, RED WOOLL...",(KNITTED UNION FLAG HOT WATER BOTTLE),0.048463,0.085106,0.037825,0.780488,9.170732,0.033701,4.167849
